In [ ]:
import time 
from ml2sql import ML2SQL

# PostgreSQL
import psycopg2 as pg
con = pg.connect(CONNECTIONSTRING)
backend = "postgres"

In [ ]:
def run_query(query, con, should_print = True):
    cursor = con.cursor()
    cursor.execute(query)
    rs = cursor.fetchall()
    if not rs:
        print("Query result is empty")
    colnames = [desc[0] for desc in cursor.description]
    if should_print:
        print(colnames)
        for res in rs:
            print(res)
    cursor.close()

def run_update_query(query, con):
    cursor = con.cursor()
    cursor.execute(query)
    con.commit()
    cursor.close()

# Prepare an example sinus timeseries

In [ ]:
table_name = "time_series_raw"
id_col_name = "id"
val_col_name = "value"
num_rows = 500

q = f"Drop table if exists {table_name}"
run_update_query(q, con)

q = f"Create table {table_name} ({id_col_name} bigint, {val_col_name} float)"
run_update_query(q, con)

values = []
for i in range(num_rows):
    values.append(f"({i})")

values_str = ",".join(values)
    
q = f"Insert into {table_name}({id_col_name}) Values {values_str}"
run_update_query(q, con)

q = f"Update {table_name} set {val_col_name}=sin(cast({id_col_name} as float) /25)"
run_update_query(q, con)

q = f"Select * from {table_name}"
#run_query(q, con)

# Example model

In [ ]:
def lstm_data_transform(x_data, y_data, num_steps=5):
    """ Changes data to the format for LSTM training 
for sliding window approach """    # Prepare the list for the transformed data
    X, y = list(), list()    # Loop of the entire data set
    for i in range(x_data.shape[0]):
        # compute a new (sliding window) index
        end_ix = i + num_steps        # if index is larger than the size of the dataset, we stop
        if end_ix >= x_data.shape[0]:
            break        # Get a sequence of data for x
        seq_X = x_data[i:end_ix]
        # Get only the last element of the sequency for y
        seq_y = y_data[end_ix]        # Append the list with sequencies
        X.append(seq_X)
        y.append(seq_y)    # Make final arrays
    x_array = np.array(X)
    y_array = np.array(y)    
    return x_array, y_array

import numpy as np

x = np.arange(0, 500, 1).reshape(-1, 1)
y = np.sin(x/25).reshape(-1, 1)

num_steps = 3
x_new, y_new = lstm_data_transform(y, y, num_steps)

train_ind = int(0.8 * x.shape[0])
x_train = x_new[:train_ind]
y_train = y_new[:train_ind]
x_test = x_new[train_ind:]
y_test = y_new[train_ind:]

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

lstm_neurons = 32

model = keras.Sequential()
model.add(layers.LSTM(lstm_neurons, input_shape=(num_steps, 1)))
model.add(layers.Dense(units=50, activation='relu'))
model.add(layers.Dense(units=1, activation='linear'))
adam = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=adam, loss='mse')
model.fit(x_train, y_train, epochs=50, verbose=0)

In [ ]:
import matplotlib.pyplot as plt
start_time = time.time()
test_predict = model.predict(x_test)
print("--- %s seconds ---" % (time.time() - start_time))
plt.style.use('ggplot')
plt.figure(figsize=(20, 7))
plt.plot(y_test, label="True value")
plt.plot(test_predict.ravel(), label="Predicted value")
plt.legend()

# Initialize ml2sql

In [ ]:
translator = ML2SQL(con, backend, model)

# Transform table

In [ ]:
ts_table_name = f"ts_table_transformed"
val_cols = translator.prepare_time_series_table(table_name, id_col_name, val_col_name, 3, ts_table_name)
q = f"Select * from {ts_table_name}"
#run_query(q, con)

# Model import

In [ ]:
model_table_name = "ts_model"
start_time = time.time()
queries = translator.model_to_relation(model_table_name)
for q in queries:
    run_update_query(q, con)
print("--- %s seconds ---" % (time.time() - start_time))
q = f"select * from {model_table_name}"
#run_query(q, con)

# Model join

In [ ]:
input_query = f"Select * from {ts_table_name}"
mj_query = translator.model_join_query(input_query, id_col_name, [val_cols], model_table_name, "prediction")
start_time = time.time()
run_query(mj_query, con)
print("--- %s seconds ---" % (time.time() - start_time))

# Raw Python

In [ ]:
W = model.layers[0].get_weights()[0]
U = model.layers[0].get_weights()[1]
b = model.layers[0].get_weights()[2]

def sigmoid(x):
    return 1 / (1 + np.exp(-x))
def tanh(x):
    return np.tanh(x)

def lstm_predict(input):
    def _compute_carry_and_output_fused(z, c_tm1):
        """Computes carry and output using fused kernels."""
        z0, z1, z2, z3 = z
        i = sigmoid(z0)
        f = sigmoid(z1)
        c = f * c_tm1 + i * tanh(z2)
        o = sigmoid(z3)
        return c, o
    
    x = input[0]
    h_tm1 = np.zeros(lstm_neurons)
    c_tm1 = np.zeros(lstm_neurons)

    for t in range(len(x)):
        z = np.dot(x[t], W)
        z += np.dot(h_tm1, U)
        z += b
        print(z)
        z = tf.split(z, num_or_size_splits=4, axis=0)
        c, o = _compute_carry_and_output_fused(z, c_tm1)
        h = o * np.tanh(c)
        h_tm1 = h
        c_tm1 = c
        print("*********************************")
    return h

# Verification and comparison

In [ ]:
input = np.reshape(x_train[6], (1, ) + x_test[0].shape)
start_time = time.time()
res = lstm_predict(input)
# Layer 2
res = res.dot(model.layers[1].get_weights()[0]) + model.layers[1].get_weights()[1]
res[res<0]=0 # relu
# Layer 3
res = res.dot(model.layers[2].get_weights()[0]) + model.layers[2].get_weights()[1]
print("Python:\t", res)
print("--- %s seconds ---" % (time.time() - start_time))
start_time = time.time()
print("TF: \t", model.predict(input)[0])
print("--- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
cursor = con.cursor()
cursor.execute(mj_query)
rs = cursor.fetchall()
print("SQL: \t", [rs[0][5]])
cursor.close()
print("--- %s seconds ---" % (time.time() - start_time))